In [2]:
import pandas as pd
import pickle
from tensorflow.keras.models import load_model
import warnings
warnings.filterwarnings('ignore')

c:\Users\Mayuresh\anaconda3\envs\myenv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


## Load The Model and Pickle Files

In [3]:
model=load_model('model.h5')

with open('onehot_enocoder_geo.pkl','rb') as file:
    onehotencoder=pickle.load(file)

with open('label_encoded_gender.pkl','rb') as file:
    labelencoder=pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)
    

In [4]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}


In [5]:
input_data_df=pd.DataFrame([input_data])

In [6]:
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [7]:
input_data_df['Gender']=labelencoder.transform(input_data_df['Gender'])
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [8]:
encoded_geo=onehotencoder.transform(input_data_df[['Geography']])

In [9]:
onehotencoder.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [10]:
geo_encoded_df=pd.DataFrame(encoded_geo.toarray(),columns=onehotencoder.get_feature_names_out(['Geography']))

In [11]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [12]:
input_data_df=pd.concat([input_data_df.drop(columns=['Geography'],axis=1),geo_encoded_df],axis=1)

In [13]:
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


## Scaling The Data

In [14]:
scaled=scaler.transform(input_data_df)
scaled

array([[-0.54204762,  0.91055421,  0.10473081, -0.68894811, -0.26230046,
         0.81966266,  0.64598061,  0.97071435, -0.88153859,  0.99828718,
        -0.57559072, -0.57779016]])

In [15]:
prediction=model.predict(scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step


array([[0.0215019]], dtype=float32)

In [16]:
prediction_prob=prediction[0][0]
if prediction_prob>0.5:
    print("The Customer is likely to churn")
else:
    print("The Customer is not likely to churn")
    

The Customer is not likely to churn
